In [1]:
import requests
from key import fred_key
import pandas as pd

base_url = "https://api.stlouisfed.org/fred/"
obs_endpoint = "series/observations"

start_date = "1950-01-01"
end_date = "2023-12-31"

d_gdp = {
    'series_id': "GDPC1",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'q',
    'units': 'log'
}
#dssdsdds
obs_param = []
obs_param.append(d_gdp)


d_cnd = {
    'series_id': "PCEND",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'q',
    'units': 'log'
}
obs_param.append(d_cnd)


d_cd = {
    'series_id': "PCEDG",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'q',
    'units': 'log'
}
obs_param.append(d_cd)





response = requests.get(base_url + obs_endpoint, params=obs_params)
# Format data
if response.status_code == 200:
    res_data = response.json()
    obs_data = pd.DataFrame(res_data['observations'])
    obs_data['date'] = pd.to_datetime(obs_data['date'])
    obs_data.set_index('date', inplace=True)
    obs_data['value'] = obs_data['value'].astype(float)

else:
    print('Failed to retrieve data. Status code:', response.status_code)




NameError: name 'obs_params' is not defined